## Answering business questions with SQL

In this project, I used the [`Chinook` database](https://github.com/lerocha/chinook-database) to answer a series of hypothetical business questions. In this notebook, I use SQLite.

### Setup

In [47]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

Below is a graphical depiction of the tables in the `Chinook` database:

In [48]:
%%html

<img src="https://s3.amazonaws.com/dq-content/191/chinook-schema.svg" alt="ChinookDB" width="500" height="600"/>

And below is a tabular summary of the tables in the `Chinook` database:

In [49]:
%%sql

SELECT 
    name
    , type
FROM sqlite_master
WHeRE type IN ('table', 'view');

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


### Which genres sell the most tracks in the USA?

In the first scenario, the Chinook record store has just signed a deal with a new record label, and our task is to find the first 3 albums that will be added to the store. We know that the record label specializes in artists from the US, as well as the artist names from the new label and their respective genres:

| Artist Name | Genre |
|-------------|-------|
| Regal | Hip-Hop |
| Red Tone | Punk |
| Meteor and the Girls | Pop |
| Slim Jim Bites | Blues |

I begin tackling this question by examining the genres that sell the most tracks in the US. The query below returns the number and percentage of tracks sold for each genre in the US:

In [50]:
%%sql

WITH track_genre AS 
(
    SELECT t.track_id
           , t.genre_id
           , g.name AS genre_name
    FROM track AS t
    LEFT JOIN genre AS g ON t.genre_id = g.genre_id
), track_invoices AS
(
    SELECT il.invoice_id
           , il.track_id
           , tg.genre_name
    FROM invoice_line AS il
    LEFT JOIN track_genre AS tg ON il.track_id = tg.track_id
), track_sales AS
(
    SELECT ti.invoice_id
           , ti.track_id
           , ti.genre_name
           , i.billing_country
           , i.total
    FROM invoice AS i
    LEFT JOIN track_invoices AS ti ON ti.invoice_id = i.invoice_id
    WHERE i.billing_country = 'USA'
), genre_totals_usa AS
(
    SELECT genre_name
           , billing_country
           , COUNT(*) AS number_of_tracks
    FROM track_sales
    GROUP BY genre_name
), sum_all_tracks AS
(
    SELECT billing_country
           , SUM(number_of_tracks) AS total
    FROM genre_totals_usa
)

SELECT gtu.genre_name
       , gtu.number_of_tracks
       , ROUND((CAST(gtu.number_of_tracks AS FLOAT) / sat.total),3) AS pct_of_all_tracks
    FROM genre_totals_usa AS gtu
    LEFT JOIN sum_all_tracks AS sat ON sat.billing_country = gtu.billing_country
    ORDER BY number_of_tracks DESC;

Done.


genre_name,number_of_tracks,pct_of_all_tracks
Rock,561,0.534
Alternative & Punk,130,0.124
Metal,124,0.118
R&B/Soul,53,0.05
Blues,36,0.034
Alternative,35,0.033
Latin,22,0.021
Pop,22,0.021
Hip Hop/Rap,20,0.019
Jazz,14,0.013


The artists from the new record label have the genres hip-hop, punk, pop, and blues. Based on the above information, our recommendation is to purchase albums from the following 3 artists:

| Artist Name | Genre |
|-------------|-------|
| Red Tone | Punk |
| Slim Jim Bites | Blues |
| Meteor and the Girls | Pop |

### Which sales support agents are performing better or worse than the others?

For the next question, we consider that each customer in the `Chinook` database gets assigned to a sales support agent when they make a purchase. We want to know if there are sales support agents that are performing better or worse than others.

To answer this question, I will analyze the purchases of customers assigned to each sales support agent, and then examine agents associated with especially high and especially low purchase amounts.

In [51]:
%%sql

WITH customer_rep AS (
    SELECT c.customer_id
           , c.support_rep_id
           , e.first_name || ' ' || e.last_name AS support_rep_name
    FROM customer AS c
    LEFT JOIN employee AS e ON c.support_rep_id = e.employee_id
), customer_totals AS (
    SELECT cr.customer_id
           , cr.support_rep_id
           , cr.support_rep_name
           , SUM(i.total) AS total_purchases
    FROM invoice AS i
    LEFT JOIN customer_rep AS cr ON i.customer_id = cr.customer_id
    GROUP BY cr.customer_id
)

SELECT support_rep_name
       , SUM(total_purchases) AS total_customer_purchases
    FROM customer_totals
    GROUP BY support_rep_name
    

Done.


support_rep_name,total_customer_purchases
Jane Peacock,1731.5099999999998
Margaret Park,1584.0000000000002
Steve Johnson,1393.92


From the result above, we can see that there is variation in the total sales facilitated by each support agent, although there are only 3 agents total.

We use the query below to check that these are indeed the only 3 support sales agents in the database:

In [52]:
%%sql

SELECT employee_id
       , first_name|| ' ' || last_name AS employee_name
   FROM employee
   WHERE title = "Sales Support Agent"
   GROUP BY employee_id;

Done.


employee_id,employee_name
3,Jane Peacock
4,Margaret Park
5,Steve Johnson


### Which countries have the highest total customers and sales?

Next, we would like to know the countries with the most customers and sales. To answer this question, for each country, I will calculate the total number of customers, total value of sales, average value of sales per customer, and average order value:

Note that there are some countries with only 1 customer. I will group these customers as "Others" for the analysis.

In [53]:
%%sql

WITH customer_invoices AS (
    SELECT c.customer_id
           , c.first_name || ' ' || c.last_name AS customer_name
           , c.country
           , i.invoice_id
           , i.total
    FROM invoice AS i
    LEFT JOIN customer AS c ON i.customer_id = c.customer_id
), customer_totals AS (
    SELECT ci.customer_id
           , ci.customer_name
           , ci.country
           , SUM(ci.total) AS customer_total
    FROM customer_invoices AS ci
    GROUP BY ci.customer_id
), country_customers AS (
    SELECT country
           , COUNT(*) AS n_customers
    FROM customer_totals
    GROUP BY country
    ORDER BY n_customers DESC
), country_customers_relabeled AS (
    SELECT 
        country
        , CASE
            WHEN country_customers.n_customers = 1 THEN 'Other'
            ELSE country_customers.country
        END AS country_name
        , n_customers
    FROM country_customers
), country_totals AS (
    SELECT ccr.country_name
           , SUM(ccr.n_customers) AS total_customers
           , SUM(ct.customer_total) AS total_sales
           , AVG(ct.customer_total) AS avg_sale_per_customer
    FROM customer_totals AS ct
    LEFT JOIN country_customers_relabeled AS ccr ON ct.country = ccr.country
    GROUP BY ccr.country_name
), country_order_avg AS (
    SELECT ccr.country_name
           , AVG(ci.total) AS avg_order_amount
    FROM customer_invoices AS ci
    LEFT JOIN country_customers_relabeled AS ccr ON ci.country = ccr.country
    GROUP BY ccr.country_name
), final_country_summary AS (
    SELECT ct.country_name
       , ct.total_customers
       , ROUND(ct.total_sales, 2) AS total_sales
       , ROUND(ct.avg_sale_per_customer, 2) AS avg_sale_per_customer
       , ROUND(coa.avg_order_amount, 2) AS avg_order_amount
    FROM country_totals AS ct
    LEFT JOIN country_order_avg AS coa ON ct.country_name = coa.country_name
)

SELECT country_name AS country
       , total_customers
       , total_sales
       , avg_sale_per_customer
       , avg_order_amount
    FROM 
    (
        SELECT fcs.*
        , CASE
            WHEN fcs.country_name = 'Other' THEN 1
            ELSE 0
        END AS sort
        FROM final_country_summary AS fcs
    )
    ORDER BY sort ASC

Done.


country,total_customers,total_sales,avg_sale_per_customer,avg_order_amount
Brazil,25,427.68,85.54,7.01
Canada,64,535.59,66.95,7.05
Czech Republic,4,273.24,136.62,9.11
France,25,389.07,77.81,7.78
Germany,16,334.62,83.66,8.16
India,4,183.15,91.57,8.72
Portugal,4,185.13,92.56,6.38
USA,169,1040.49,80.04,7.94
United Kingdom,9,245.52,81.84,8.77
Other,15,1094.94,73.0,7.45


The results indicate that the USA has the highest number of total customers, as well as the highest total sales amount (aside from countries collectively marked as Other). Czech Republic has the highest average total sales per customer, as well as the highest average order amount.

### What percentage of purchases are individual tracks vs. whole albums?

To answer this question, I need to identify whether each invoice has all the tracks from an album, which I do in the queries below.

First, I create a table that has all the tracks and their corresponding album IDs included in each invoice:

In [54]:
%%sql

WITH invoices_tracks_albums AS (
    SELECT i.invoice_id
           , a.album_id
           , t.track_id
    FROM invoice AS i
    LEFT JOIN invoice_line AS il ON i.invoice_id = il.invoice_id
    LEFT JOIN track AS t ON il.track_id = t.track_id
    LEFT JOIN album AS a ON t.album_id = a.album_id
)

SELECT *
    FROM invoices_tracks_albums
    ORDER BY invoice_id, album_id, track_id
    LIMIT 10

Done.


invoice_id,album_id,track_id
1,91,1158
1,91,1159
1,91,1160
1,91,1161
1,91,1162
1,91,1163
1,91,1164
1,91,1165
1,91,1166
1,91,1167


I also need a table that lists the tracks on each album:

In [55]:
%%sql

WITH album_tracks AS (
    SELECT a.album_id
           , t.track_id
    FROM track AS t
    LEFT JOIN album AS a ON t.album_id = a.album_id
    WHERE a.album_id IS NOT NULL 
    ORDER BY a.album_id
)

SELECT *
    FROM album_tracks
    ORDER BY album_id, track_id
    LIMIT 10

Done.


album_id,track_id
1,1
1,6
1,7
1,8
1,9
1,10
1,11
1,12
1,13
1,14


Next, we assume that if an invoice contains N number of tracks from a given album, and we know that album has N tracks, then the invoice includes the entire album. An invoice with a different number of tracks did not include the entire album.

So, I need another table showing the count of tracks per album:

In [56]:
%%sql

WITH album_tracks AS (
    SELECT a.album_id
           , t.track_id
    FROM track AS t
    LEFT JOIN album AS a ON t.album_id = a.album_id
    WHERE a.album_id IS NOT NULL 
    ORDER BY a.album_id
), album_track_counts AS (
    SELECT album_id
           , COUNT(*) AS n_tracks
    FROM album_tracks
    GROUP BY album_id
)

SELECT *
    FROM album_track_counts
    LIMIT 10

Done.


album_id,n_tracks
1,10
2,1
3,3
4,8
5,15
6,13
7,12
8,14
9,8
10,14


Next, I need to count the number of tracks purchased on each album, on each invoice:

In [57]:
%%sql

WITH invoices_tracks_albums AS (
    SELECT i.invoice_id
           , a.album_id
           , t.track_id
    FROM invoice AS i
    LEFT JOIN invoice_line AS il ON i.invoice_id = il.invoice_id
    LEFT JOIN track AS t ON il.track_id = t.track_id
    LEFT JOIN album AS a ON t.album_id = a.album_id
), invoice_album_counts AS (
    SELECT invoice_id
           , album_id
           , COUNT(*) AS n_tracks_bought
    FROM invoices_tracks_albums
    GROUP BY invoice_id, album_id
)

SELECT *
    FROM invoice_album_counts
    ORDER BY invoice_id, album_id
    LIMIT 10

Done.


invoice_id,album_id,n_tracks_bought
1,91,16
2,20,1
2,34,1
2,39,1
2,66,1
2,73,1
2,134,1
2,190,1
2,215,1
2,218,1


Finally, I need to compare the table of actual tracks per album, with the table of purchased tracks per album and invoice. For each invoice/album combination, I assign a value of 1 to the new variable `full_album_purchased` if the numbers match, and I assign 0 if they do not:

In [58]:
%%sql

WITH invoices_tracks_albums AS (
    SELECT i.invoice_id
           , a.album_id
           , t.track_id
    FROM invoice AS i
    LEFT JOIN invoice_line AS il ON i.invoice_id = il.invoice_id
    LEFT JOIN track AS t ON il.track_id = t.track_id
    LEFT JOIN album AS a ON t.album_id = a.album_id
), invoice_album_counts AS (
    SELECT invoice_id
           , album_id
           , COUNT(*) AS n_tracks_bought
    FROM invoices_tracks_albums
    GROUP BY invoice_id, album_id
), album_tracks AS (
    SELECT a.album_id
           , t.track_id
    FROM track AS t
    LEFT JOIN album AS a ON t.album_id = a.album_id
    WHERE a.album_id IS NOT NULL 
    ORDER BY a.album_id
), album_track_counts AS (
    SELECT album_id
           , COUNT(*) AS n_tracks
    FROM album_tracks
    GROUP BY album_id
), invoice_album_summary AS (
    SELECT iac.invoice_id
       , iac.album_id
       , iac.n_tracks_bought
       , atc.n_tracks
       , CASE 
            WHEN iac.n_tracks_bought = atc.n_tracks THEN 1
            ELSE 0
            END 
         AS full_album_purchase
            
    FROM invoice_album_counts AS iac
    LEFT JOIN album_track_counts AS atc ON iac.album_id = atc.album_id
)

SELECT *
    FROM invoice_album_summary
    LIMIT 15

Done.


invoice_id,album_id,n_tracks_bought,n_tracks,full_album_purchase
1,91,16,16,1
2,20,1,11,0
2,34,1,17,0
2,39,1,21,0
2,66,1,10,0
2,73,1,30,0
2,134,1,10,0
2,190,1,16,0
2,215,1,14,0
2,218,1,15,0


We can see from the above table that some invoices included tracks purchased from each album. In order to group things by invoice, we only count an invoice as a full-album purchase if it includes all tracks from all albums on the invoice:

In [59]:
%%sql

WITH invoices_tracks_albums AS (
    SELECT i.invoice_id
           , a.album_id
           , t.track_id
    FROM invoice AS i
    LEFT JOIN invoice_line AS il ON i.invoice_id = il.invoice_id
    LEFT JOIN track AS t ON il.track_id = t.track_id
    LEFT JOIN album AS a ON t.album_id = a.album_id
), invoice_album_counts AS (
    SELECT invoice_id
           , album_id
           , COUNT(*) AS n_tracks_bought
    FROM invoices_tracks_albums
    GROUP BY invoice_id, album_id
), album_tracks AS (
    SELECT a.album_id
           , t.track_id
    FROM track AS t
    LEFT JOIN album AS a ON t.album_id = a.album_id
    WHERE a.album_id IS NOT NULL 
    ORDER BY a.album_id
), album_track_counts AS (
    SELECT album_id
           , COUNT(*) AS n_tracks
    FROM album_tracks
    GROUP BY album_id
), invoice_album_summary AS (
    SELECT iac.invoice_id
       , iac.album_id
       , iac.n_tracks_bought
       , atc.n_tracks
       , CASE 
            WHEN iac.n_tracks_bought = atc.n_tracks THEN 1
            ELSE 0
            END 
         AS full_album_purchase

    FROM invoice_album_counts AS iac
    LEFT JOIN album_track_counts AS atc ON iac.album_id = atc.album_id
), invoice_summary AS (
    SELECT invoice_id
           , COUNT(album_id) AS albums_on_invoice
           , SUM(full_album_purchase) AS full_album_purchases
           , CASE
                WHEN COUNT(album_id) = SUM(full_album_purchase) THEN 1
                ELSE 0
                END
             AS all_albums_purchased
    FROM invoice_album_summary
    GROUP BY invoice_id
)

SELECT *
    FROM invoice_summary
    LIMIT 10

Done.


invoice_id,albums_on_invoice,full_album_purchases,all_albums_purchased
1,1,1,1
2,10,0,0
3,2,0,0
4,8,1,0
5,1,1,1
6,2,0,0
7,11,1,0
8,10,0,0
9,9,0,0
10,2,0,0


Now that we have final identifiers for each invoice indicating whether it was a true full-album purchase, I can aggregate over the resulting table to calculate the number and percentage of invoices that were full album purchases:

In [60]:
%%sql

WITH invoices_tracks_albums AS (
    SELECT i.invoice_id
           , a.album_id
           , t.track_id
    FROM invoice AS i
    LEFT JOIN invoice_line AS il ON i.invoice_id = il.invoice_id
    LEFT JOIN track AS t ON il.track_id = t.track_id
    LEFT JOIN album AS a ON t.album_id = a.album_id
), invoice_album_counts AS (
    SELECT invoice_id
           , album_id
           , COUNT(*) AS n_tracks_bought
    FROM invoices_tracks_albums
    GROUP BY invoice_id, album_id
), album_tracks AS (
    SELECT a.album_id
           , t.track_id
    FROM track AS t
    LEFT JOIN album AS a ON t.album_id = a.album_id
    WHERE a.album_id IS NOT NULL 
    ORDER BY a.album_id
), album_track_counts AS (
    SELECT album_id
           , COUNT(*) AS n_tracks
    FROM album_tracks
    GROUP BY album_id
), invoice_album_summary AS (
    SELECT iac.invoice_id
       , iac.album_id
       , iac.n_tracks_bought
       , atc.n_tracks
       , CASE 
            WHEN iac.n_tracks_bought = atc.n_tracks THEN 1
            ELSE 0
            END 
         AS full_album_purchase

    FROM invoice_album_counts AS iac
    LEFT JOIN album_track_counts AS atc ON iac.album_id = atc.album_id
), invoice_summary AS (
    SELECT invoice_id
           , COUNT(album_id) AS albums_on_invoice
           , SUM(full_album_purchase) AS full_album_purchases
           , CASE
                WHEN COUNT(album_id) = SUM(full_album_purchase) THEN 1
                ELSE 0
                END
             AS all_albums_purchased
    FROM invoice_album_summary
    GROUP BY invoice_id
)

SELECT CASE
             WHEN all_albums_purchased = 1 THEN 'Full album(s) purchased'
             ELSE 'Individual track(s) purchased'
             END AS purchase_type
       , COUNT(*) count_invoices
       , ROUND(100*CAST(COUNT(all_albums_purchased) AS FLOAT) / (
           SELECT COUNT(*) FROM invoice_summary), 1) || '%' AS percentage_invoices 
    FROM invoice_summary
    GROUP BY all_albums_purchased

Done.


purchase_type,count_invoices,percentage_invoices
Individual track(s) purchased,500,81.4%
Full album(s) purchased,114,18.6%


Based on these results, 81.4% of invoices represented individual track(s) being purchased, whereas 18.6% of all invoices represented full album(s) being purchased. Therefore, keeping both options available (buying individual tracks from albums as well as full albums) seems important.